### Segment - Monolithic to Microservices to Monolithic Architecture

**Core services**
- it delivers messages to third-party endpoints
- retries messages upon failure
- archives any undelivered messages

**why not just use a queue here? Building a fully distributed job scheduler seems a bit overdone**
- At segment, Kafka was already used extensively 
- it passes nearly 1M messages per second through it
- it’s been the core building block of all of our streaming pipelines

**When does a queue break down**
- The problem with using any sort of queue is that you are fundamentally limited in terms of how you access data. 
- a queue only supports two operations (push and pop)

<img src="https://images.ctfassets.net/i8bfc4nr05rq/526JE4CsskIje2tIoBxmbr/c8108e96a2f99d5d8f1499e08a43e635/asset_sJlQSUXb.png" width=600 height=400>
     
**a single queue architecture**
<img src="https://images.ctfassets.net/i8bfc4nr05rq/1TMg75tMuAumjmYHu5UvDP/a59d8315041ceb7a48da16c6cdfb16d9/asset_VdRJbhUe.png" width=600 height=400>


**What happens when single endpoint gets slow**
<img src="https://images.ctfassets.net/i8bfc4nr05rq/3wQyQCVvbIwDOS3roQmaUv/ec97341d39dabd25098e627f47028cbc/asset_QVaGLpel.png" width=600 height=400>


**queues per destination architecture**
- updated queueing topology to route events into separate queues based upon the downstream endpoints they would hit
- router was added in front of each queue, which would only publish messages to a queue destined for a specific API endpoint
- segment is a large, multi-tenant system, so some sources of data will generate substantially more load than others

<img src="https://images.ctfassets.net/i8bfc4nr05rq/3mwwfjAAWWVLMmqZhKcEEH/97e2569dfc3ca1fedc9f5da233ce94f3/asset_DLWTCwfw.png" width=600 height=400>


- https://segment.com/docs/guides/
- https://segment.com/blog/goodbye-microservices/
- https://segment.com/blog/introducing-centrifuge/

### Istio
https://blog.christianposta.com/microservices/istio-as-an-example-of-when-not-to-do-microservices/


### LinkedIn

#### Starting days

https://engineering.linkedin.com/architecture/brief-history-scaling-linkedin

Leo - single monolithic application doing it all. That single app was called Leo. It hosted web servlets for all the various pages, handled business logic, and connected to a handful of LinkedIn databases

Cloud - manage member to member connections. We needed a system that queried connection data using graph traversals and lived in-memory for top efficiency and performance. to scale independently of Leo, so a separate system for our member graph called Cloud was born - LinkedIn’s first service

Lucene - member graph service started feeding data into a new search service running Lucene.

![Master Slave Model]()
<img src="https://content.linkedin.com/content/dam/engineering/en-us/blog/migrated/arch_master_slave_0.png" width=600 height=400>

### How do Pokemon Go scale to millions of request
- https://www.scaleyourapp.com/distributed-systems-scalability-part1-heroku-client-rate-throttling/
- https://cloud.google.com/blog/topics/developers-practitioners/how-pok%C3%A9mon-go-scales-millions-requests


### How does Uber scale to millions of concurrent requests

- Fulfillment platform
  - goes through series of checks and balances and then triggers fulfillment to create new order for consumer
  - order is the intent of consumer
  - intent is then translated into jobs that needs to be processed
  - this information is stored in spanner
  - these jobs are read by matching engine
  - offered to any open provider or supplier nearby
  - earlier all of this data was saved in **on-prem database** and recently moved to **Spanner**
  
<img src="./images/uberRequestFlow.png">

- Challenges with on-prem database
  - No-SQL storage engine **Cassandra** was used to save all real time fulfillment entities
  - to maintain serializability on top of Cassandra, **Ringpop** was used to provide individual **entity-level serialization**
  - Challenges with NoSQL
    - Cassandra follows the principle of **eventual consistency**
    - there is **no guarantee of low-latency quorum writes with Cassandra**
    - witnessed complex storage interactions that required multi-row and multi-table writes.
    - to tackle this, an application layer framework was build to orchestrate this operation using saga pattern
    - these inconsistencies had to be manually corrected
    - in real world - two different drivers are dispatched to a customer
  - Horizontal scaling bottlenecks were observed in overall architecture
    - this was primarily because how application layer was sharded with Ringpop
    - properties of traditional SQL-based storage ACID guarantees were observed
    - consistency was the primary requirement
    - NoSQL requirements with strong ACID guarantees were needed
- Google Spanner was the solution
  - what were the explorations for transitioning from NoSQL to NewSQL
    - Consistency with high resilency and availability were the requirement criterias
    - CockroachDB, FoundationDB and Cloud Spanner were considered
    - functional requirements were fulfilled
      - scaled horizontally with our benchmarks, and provided a managed solution for cluster management and maintenance
    - what is the new architecture
      - every user request results in a transaction against one or more rows and across one or more tables in cloud Spanner
      - consistent view of data is available both to internal and external customers

<img src="./images/uberNewFulfillmentArchitecture.png">

- latency is the biggest factor, what is the networking architecture looks like 
  - resilient and highly reliable infrastructure to support Uber's workload
  - networking infrastructure was divided into 2 major components
    - physical layer consisting of interconnections between Uber and Cloud vendors
    - logical layer consisting of virtual connections on top of physical layer to achieve redundancy
      - this is achieved by making multiple routers and having local access points at each physical network route
      - leverage Google's private Google access to route using cloud interconnect VLAN attachments
      - reduced the need of routing traffic through public internet and provided additional reliability
      - benchmarking tests were designed     
  
<img src="./images/uberLowLatencyNetwork.png">

- how was the migration and transition made seemless
  - database topology was significant different from old platform, any kind of live migration of data was ruled out
  - most of the data is ephemeral and changing continuously, backups would have resulted in loss of data
  - systems were built that will intercept request from user session and attach to the new user session 
  - till the past order is not complete, this migration is not done
  - only open user session with no active orders were migrated to new architecture
  - tooling were tested rigorously in testing, staging and shadow environments
  - test cities were developed to check migrations with one city at a time
  - continuously migrate over 6 month period

  
- Performance Optimization
  - networking stack
    - TCP Reset
      - the spanner session needs to be re-established for the forwarded requests
      - by optimizing gRPC channel pool whenever a channel is affected by TCP resets, requests are automatically forwarded temporarily to backup empty gRPC channel
    - Intermittent packet loss
      - with frequent health checks, broken TCP connections can be detected within 5 sec and graceful connections can be triggered
  
- To take advantage of cloud Spanner elasticity, autoscaler was built
  - constantly tunes the number of nodes based on CPU utilization target
  - as traffic is variable based on constant CPU utilization target, maximum elasticity is achieved from Cloud spanner cluster 
  
<img src="./images/uberAutoscaler.png">

- On-prem cache was made to handle workload which is very read-heavy
  - improves latency and cost
  - only route stale reads to Spanner 
  - allow cache to serve snapshot isolation based on Spanner's queue time

<img src="./images/uberOnpremcache.png">



- READ:
  - "How Ringpop from Uber Engineering helps distribute your application"
  - Saga pattern - in microservices architecture it tracks all events of distributed transactions as a sequence and decides the rollbacks events in case of a failure
  - Planet scale
  - Strong read and Stale read
  

### What database does Facebook use
- MySQL is the primary database used for storing all social data
- Memcache is used in addition to MySQL as caching layer
- Cassandra for search



To manage BigData Facebook leverages Apache Hadoop, HBase, Hive, Apache Thrift and PrestoDB. All these are used for data ingestion, warehousing and running analytics.

Apache Cassandra is used for the inbox search

Beringei & Gorilla, high-performance time-series storage engines are used for infrastructure monitoring. 

LogDevice, a distributed data store is used for storing logs. 

### How is Git different from any other source systems

### How does Google Ad business model works

### How to create a microservice architecture with Google Cloud - Nylas

- Workflow Automation 
  - by collecting data from multiple applications 
  - compiling insights
  - create end-to-end automation workflow

- handle communication data driving business process automation

- Problems to tackle 
  - security
  - scalability
  - performance
  - cost
  
- Why Google Cloud
  - best distributed technology
  - GKE - Google Kubernetes Engine is the best cloud provider to run Kubernetes workload securely at scale 
  - one of the most performant data stores in Spanner
  
- Architecture  
  - the advantage of using **GKE** is its ability to **handle the container orchestration**, **Cloud Pub/Sub for message bus** and **Cloud Spanner for relational data store**   
  - Spanner processes 1B requests per second at peak
  - latency of less than 7 milliseconds
  - GCP services are very robust

<img src="./images/nylasOldArchitecture.png">

- Transition 
  - from Python based application to Go - code rewrite
    - services were rewritten in Go 
    - 10x throughput improvement was observed
  - move from virtual machine instances on AWS EC2 to GKE 
    - move the orchestration from **AWS autoscaler group to GKE**
    - GKE was used to orchestrate containers 
    - due to security reasons, nodes have to be cycled very quickly
    - were able to have upto 15000 nodes in Kubernetes cluster 
    - other cloud providers dont provide this level of support
    - **gVisor** is used to run containers to create strong isolation between application and operating system
    - helps lock down host memory and storage access and enforce least permission principle at operating system level
    - **gVisor is an application kernel**, written in Go, that implements substantial portion of Linux system call interface. It provides an additional layer of isolation between running applications and host operating system    
    
- 20TB of data processing in MySQL shards
  - Cloud spanner was used to keep application states 
  - store keys for each accounts that needs to be connected
  - fast read/write is extremely critical for large number of calls
  - high performance to have predictable end-to-end latency

<img src="./images/nylasNewArchitecture.png">

- PII(Personally Identifiable Information)
  - Nylas does not have to hold PII
  - for security services
  
- Migrated from database centric design to event based design
  - message bus is the heart of all application
  - to make this transition, Google Pub/Sub was choosen
  - Google Pub/Sub allows services to communicate asynchronously with latencies on the order of 100 milliseconds
  - in legacy AWS architecture which was more of data driven architecture
    - the data is written into **Dynamo**
    - then gets converted by **lambda function**
    - then goes to **Kinesis event stream**
  - in new architecture, this is simplified
    - event it first
    - rewrote it directly on Google Pub/Sub
    - puts lot of strain on system, with trust that this will get scale and perform reliably and consistently
    - on **Google Pub/Sub, the performance latency is minimal**
    - event bus can act as data store
    
- Benefits of migrating
  - **30 times performance on each node** was observed
  - **Elasticity of architecture** is amazing
  

### How to architect an AI/ML powered healthcare platform on Google Cloud - Vida Health
- provides comprehensive virtual care solution tailored to personalized health goals and preferences
- what are the challenges in building such application
  - data coming from different sources of different types 
  - needs to be integrated seemlessly
  - both for mobile and web users
  
- Why Google cloud
  - 50% cost reduction
  - developer productivity, one single place to look at
  - focus on AI/ML
  
- Challenges with ML and AI
  - make provider more efficient at their jobs
  - use NLP and speech to text - take down notes for doctors/assistants
  - how to get more patients
    - depending on severity and acuity of patients 
    - give signals to providers
    - prioritize such cases 
    
- Vidapedia - major recommendation platform - using Workspace and BigQuery. How does it work?
  - Vidapedia 
    - set of protocols, as Google docs, written to help providers/clinicians 
    - 200 different protocols are created, not feasible to remember/memorize
    - how to get such information at high speed
  - Vidapedia recommendation system does on machine learning
    - look into patient-provider interaction going on
    - surface right protocol for such interaction
    - ML model detects and triggers particular protocol during or after the session
    - provider is able to give right care to patient
  - this was possible using Google Doc protocols interaction with Google Workspace 
    - use this data, index them
    - store it in BigQuery
    - build ML tools on top of that
    
- Data pipeline architecture
  - data comes from different sources
  - providers provide consultation notes
  - derive insights 
  - data gets ingested both in relational and unstructured format
  - all these data is saved in BigQuery
  - which is then used in analytics application deriving insights
  - visualization tools such as data studio and looker
  - ml models are built which is then sourced into visualization tools
  - exploration in terms of VertexAI for quickly building these models and deploying them
  
<img src="./images/vidaDataAnalyticsArchitecture.png">

- Transformations
  - in past custom ETL pipelines were designed, which is a challenge
  - currently **dbt** is used for transformations
  
- how is the orchestration done
  - **Cloud Composer** is used for orchestration the pipeline, scheduling all of this ingest, processing data, and running all the stuff

- how is security requirement resolved
  - patient data is very sensitive
  - stringent privacy laws
  - use **Google Data Loss Protection(DLP)**
  - **mask PII data**
  - reduce surface area
  
- what is the Web Architecture designed
  - SADA practices - well known in the industry
  - standard 3-tier architecture
  - load balancers at the front
  - deployed in Google Kubernetes engine
  - middle API layer that returns all the request response
  - at the end there is GCS Postgres database, scaling is easy 
  - easy to scale
  
<img src="./images/vidaWebArchitecture.png">

- What next
  - Google Cloud Healthcare API
    - provides managed solution for storing and accessing healthcare data in Google cloud
  - Google FHIR - Fast Healthcare Interoperability Resources
    - Google fi